In [14]:
import os
import sys
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import cv2 as cv

from keras.layers import Flatten, Dense
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import MobileNet, preprocess_input
from keras.losses import categorical_crossentropy
from keras.utils import img_to_array
from keras.utils.image_utils import smart_resize

import tensorflow as tf

print(f'Python: {sys.version}')

Python: 3.10.4 (tags/v3.10.4:9d38120, Mar 23 2022, 23:13:41) [MSC v.1929 64 bit (AMD64)]


In [3]:
RANDOM_STATE = 0

In [4]:
# Working with pre trained model 

#base_model = MobileNet(input_shape=(224, 224, 3), include_top=False)
base_model = MobileNet(
    input_shape=None,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
)


for layer in base_model.layers:
  layer.trainable = False


x = Flatten()(base_model.output)
x = Dense(units=26 , activation='softmax')(x)

# creating our model.
model = Model(base_model.input, x)

In [5]:
model.compile(optimizer='adam', loss=categorical_crossentropy , metrics=['accuracy'])

In [95]:
df_treino = pd.read_csv("base_aus.csv")
df_treino = df_treino.drop(columns="Unnamed: 0")
df_treino = pd.concat([df_treino]).sample(frac=1)

df_treino

,emotion,img,aus
13217,fear,./data/images/images/train/fear/22611.jpg,"5,20"
28173,neutral,./data/images/images/train/neutral/5540.jpg,0
21055,sad,./data/images/images/train/sad/24269.jpg,"6,15,54,64"
2532,angry,./data/images/images/train/angry/30484.jpg,"4,5,7,10,22,23,25,26"
1609,angry,./data/images/images/train/angry/229.jpg,"4,5,7,17,24"
...,...,...,...
21136,sad,./data/images/images/train/sad/24710.jpg,"6,15,54,64"
4406,disgust,./data/images/images/train/disgust/8675.jpg,"10,16,25,26"
22846,sad,./data/images/images/train/sad/35661.jpg,"11,17"
4354,disgust,./data/images/images/train/disgust/4969.jpg,"10,16,25,26"


In [98]:
labels = np.sort(df_treino['aus'].unique())
labels

array(['0', '1,2,26', '1,2,4,5', '1,2,4,5,20,25', '1,2,4,5,25', '1,2,5',
       '1,2,5,26', '1,2,5,27', '1,4,11,15,54,64', '1,4,15', '1,4,15,17',
       '10, 17', '10,16,25,26', '11,17', '4,5,7,10,22,23,25,26',
       '4,5,7,10,23,25,27', '4,5,7,17,23', '4,5,7,17,24', '4,5,7,23',
       '5,20', '5,26', '5,27', '6,12', '6,15,54,64', '9,16,15,26', '9,17'],
      dtype=object)

In [72]:
train_datagen = ImageDataGenerator(
    zoom_range = 0.2, 
    shear_range = 0.2, 
    horizontal_flip=True, 
    rescale = 1./255
)
train_data = train_datagen.flow_from_dataframe(
    dataframe = df_treino,
    directory=".",
    x_col = "img",
    y_col = "aus",
    subset = "training",
    batch_size = 32,
    seed = 42,
    class_mode = "categorical",
    target_size = (48, 48)
)

Found 28821 validated image filenames belonging to 26 classes.


In [49]:
hist = model.fit(
    train_data,
    steps_per_epoch=10,
    epochs=100,
    validation_data=train_data,
    validation_steps=8,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(
            filepath='./results/model_aus.h5',
            save_best_only=True,
            monitor="accuracy"
        )
    ]
)

print()
max_acc = max(hist.history['accuracy']) * 100
print(f'Accuracy: {max_acc = :.2f}%')

Epoch 1/100
10/10 [==============================] - 1s 89ms/step - loss: 2.7375 - accuracy: 0.2562 - val_loss: 2.7894 - val_accuracy: 0.2383
Epoch 2/100
10/10 [==============================] - 1s 83ms/step - loss: 2.6627 - accuracy: 0.2750 - val_loss: 2.7406 - val_accuracy: 0.2500
Epoch 3/100
10/10 [==============================] - 1s 71ms/step - loss: 2.7140 - accuracy: 0.2625 - val_loss: 2.6715 - val_accuracy: 0.2969
Epoch 4/100
10/10 [==============================] - 1s 74ms/step - loss: 2.7962 - accuracy: 0.2406 - val_loss: 2.7918 - val_accuracy: 0.2109
Epoch 5/100
10/10 [==============================] - 1s 73ms/step - loss: 2.7557 - accuracy: 0.2625 - val_loss: 2.7666 - val_accuracy: 0.2383
Epoch 6/100
10/10 [==============================] - 1s 85ms/step - loss: 2.6845 - accuracy: 0.2781 - val_loss: 2.7867 - val_accuracy: 0.2188
Epoch 7/100
10/10 [==============================] - 1s 85ms/step - loss: 2.6582 - accuracy: 0.3000 - val_loss: 2.7461 - val_accuracy: 0.2148
Epoch 

steps_per_epoch=10,
epochs=30,
Accuracy: max_acc = 30.94%


steps_per_epoch=30,
epochs=30,
Accuracy: max_acc = 26.77%


steps_per_epoch=10,
epochs=100,
Accuracy: max_acc = 31.25%


steps_per_epoch=100,
epochs=100,
Accuracy: max_acc = 26.88%


steps_per_epoch=450,
epochs=10,
Accuracy: max_acc = 25.15%

In [109]:
img_cv = cv.imread('./data/images/images/validation/happy/30.jpg', cv.IMREAD_COLOR)
img_keras = img_to_array(img_cv)
img_keras = np.expand_dims(img_keras, axis=0)
img_keras = smart_resize(img_keras, (224, 224))
prediction = model.predict(img_keras)[0]
mean = prediction.mean()
std = prediction.std()
# prediction_classes = np.where(prediction > mean + std)[0]
# aus = {}
# for pred in prediction_classes:
#     aus[int(pred)] = float(prediction[pred])
prediction_idx = np.argmax(prediction)
prediction_label = labels[prediction_idx]
prediction_prob = prediction[prediction_idx]

print(f'{prediction_label}: {round(prediction_prob * 100, 2)}%')

1/1 [==============================] - 0s 28ms/step
6,12: 22.97%


In [63]:
len(prediction)

26

In [97]:
labels

array(['0', '1,2,26', '1,2,4,5', '1,2,4,5,20,25', '1,2,4,5,25', '1,2,5',
       '1,2,5,26', '1,2,5,27', '1,4,11,15,54,64', '1,4,15', '1,4,15,17',
       '10, 17', '10,16,25,26', '11,17', '4,5,7,10,22,23,25,26',
       '4,5,7,10,23,25,27', '4,5,7,17,23', '4,5,7,17,24', '4,5,7,23',
       '5,20', '5,26', '5,27', '6,12', '6,15,54,64', '9,16,15,26', '9,17'],
      dtype=object)